In [1]:
!pwd

/home/yorshula/yorshula_master


In [2]:
from lra.layers import *
from lra.models import *
from lra.setups import *
from lra.utils  import *

import sys
sys.path.append('/home/yorshula/long-range-arena')


from lra_benchmarks.matching.input_pipeline import get_matching_datasets

In [3]:
train_dataset, valid_dataset, test_dataset, encoder = get_matching_datasets(1, None, tokenizer='char', data_dir='/home/yorshula/lra_release/lra_release/tsv_data', batch_size=BPE_MATCH_SETUP['batch_size'], max_length=BPE_MATCH_SETUP['max_length'])

INFO:tensorflow:/home/yorshula/lra_release/lra_release/tsv_data/new_aan_pairs.train.tsv


2022-05-02 13:38:44.104118: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


INFO:tensorflow:/home/yorshula/lra_release/lra_release/tsv_data/new_aan_pairs.eval.tsv
INFO:tensorflow:/home/yorshula/lra_release/lra_release/tsv_data/new_aan_pairs.test.tsv


2022-05-02 13:38:44.106400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 13:38:44.150964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 13:38:44.153047: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 13:38:44.154822: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 13:38:44.156536: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

INFO:tensorflow:Finished getting dataset.


INFO:tensorflow:Finished getting dataset.


In [4]:
BPE_MATCH_SETUP['device'] = 'cuda:1'
#LISTOPS_SETUP['model_type'] = MatchingTransformerSkip

def att_factory(hidden_dim, qkv_dim, num_heads, dropout_rate):
    
    lka = nn.Sequential(
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.0),
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.0),
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Softplus(), False, LAMBDA=0.0),
        
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.1),
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.1),
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Softplus(), False, LAMBDA=0.1)

        #HeadWiseFF(num_heads, qkv_dim, dropout_rate, nn.Softplus(), use_bias=False, LAMBDA=0.1),   
    )
    return LKAAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, lka)
    
    #return FtAttention(hidden_dim, qkv_dim, num_heads, dropout_rate)
    #return SimpleAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, use_lin=True)
    #return SimpleAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, use_lin=False)

model, criterion, optimizer, schedule_func, scheduler = training_setup(BPE_MATCH_SETUP, encoder.vocab_size, att_factory)

In [6]:
import torch

BPE_MATCH_SETUP['device'] = 'cuda:1'
#BPE_MATCH_SETUP['steps'] = 20
#BPE_MATCH_SETUP['eval_period'] = 10

test_accuracy = [  ]

for i in range(1): ####!!!!!!!!!!!!!!
  path = 'model_to_test_' + str(i) + '.b'

  model, criterion, optimizer, schedule_func, scheduler = training_setup(BPE_MATCH_SETUP, encoder.vocab_size, att_factory)

  checkpoint = train_matching_model(BPE_MATCH_SETUP, model, path, train_dataset, valid_dataset, optimizer, criterion, scheduler)
  model.load_state_dict(checkpoint['model_state_dict'])
  
  _, _, acc = matching_test(model, criterion, test_dataset, BPE_CLS_SETUP['device'])
  test_accuracy.append(acc)

test_accuracy = np.mean(test_accuracy)

print(f'\nTotal accuracy: {test_accuracy:.4f}')

Epoch 0


2022-05-02 13:42:49.000637: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 1270 of 2048
2022-05-02 13:42:53.181697: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


[====================] 10/10: - running_loss: 0.6936 - running_reg: 0.000000 - running_acc: 0.5242 - lr: 0.00000 - epoch_loss: 0.6937 - epoch_reg: 0.000000 - epoch_acc: 0.5188 - valid_loss: 0.6932 - valid_reg: 0.000000 - valid_acc: 0.5045 - epoch_time: 326.8389 s
Epoch 1
[====================] 10/10: - running_loss: 0.6948 - running_reg: 0.000000 - running_acc: 0.4552 - lr: 0.00000 - epoch_loss: 0.6951 - epoch_reg: 0.000000 - epoch_acc: 0.4469 - valid_loss: 0.6932 - valid_reg: 0.000000 - valid_acc: 0.5045 - epoch_time: 305.2100 s
 - test_loss: 0.6931 - test_reg: 0.000000 - test_acc: 0.5061 - test_time: 282.7273 s

Total accuracy: 0.5061


In [7]:
model

MatchingTransformer(
  (embed_layer): TEmbedding(
    (embedding): Embedding(257, 128, padding_idx=0)
  )
  (blocks): ModuleList(
    (0): TBlock(
      (layernorm_input): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (layernorm_inter): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (attention): LKAAttention(
        (q): Linear(in_features=128, out_features=128, bias=True)
        (k): Linear(in_features=128, out_features=128, bias=True)
        (v): Linear(in_features=128, out_features=128, bias=True)
        (lka): Sequential(
          (0): AMGOLU(
            (orth_weight): HWLinear()
            (gate_weight_a): HWLinear()
            (gate_weight_b): HWLinear()
            (kernel_nonlinearity): Identity()
            (gate_nonlinearity): Sigmoid()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): AMGOLU(
            (orth_weight): HWLinear()
            (gate_weight_a): HWLinear()
            (gate_weight_b): HWLinear()
 